In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import matplotlib.pyplot as plt

A_to_au_conversion = 1.8897259885789

In [2]:
def calc_adapt_vqe_N2H4(threshold, active_electrons=4, active_orbitals=4):
    symbols = ["N", "H", "H", "N", "H", "H"]
    # ground state coordinate: 
    geometry = np.array([0.0, 0.0, 0.0 , 0.0, 0.0, 1.015264, 0.978541, 0.0, -0.270591, -0.627449, 1.276052, -0.477492 , -0.897827, 1.825923, 0.332013 , 0.080714, 1.825923, -0.953842])*A_to_au_conversion

    H, qubits = qchem.molecular_hamiltonian(
        symbols,
        geometry,
        active_electrons=active_electrons,
        active_orbitals=active_orbitals,
        # method="pyscf"
    )
    print(H)

    active_electrons = active_electrons

    singles, doubles = qchem.excitations(active_electrons, qubits)

    print(f"Total number of excitations = {len(singles) + len(doubles)}")
    singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
    doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
    operator_pool = doubles_excitations + singles_excitations   
    hf_state = qchem.hf_state(active_electrons, qubits)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit():
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
        return qml.expval(H)
    energy_array = []
    opt = qml.optimize.AdaptiveOptimizer()
    for i in range(len(operator_pool)):
        circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool)
        energy_array.append(energy)
        # if i % 3 == 0:
        print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
        # print(qml.draw(circuit, decimals=None)())
        print()
        if gradient < threshold*10^(-threshold):
            break
    return energy_array, circuit


In [3]:
config = [[4,4]]
threshold = 3

for i in range(len(config)):
    print("Configuration: ", threshold, config[i][0], config[i][1])
    E, circuit = calc_adapt_vqe_N2H4(threshold, active_electrons=config[i][0], active_orbitals=config[i][1])
    
    file_path = f"data/N2H4_{threshold}_{config[i][0]}_{config[i][1]}.txt"
    # Open the file in write mode
    with open(file_path, "w") as file:
        # Write each element of the array to a new line
        for element in E:
            file.write(str(element) + "\n")

Configuration:  3 4 4
  (-108.2509117996428) [I0]
+ (-0.22725040055629014) [Z7]
+ (-0.22725040055629003) [Z6]
+ (-0.14333511784309017) [Z5]
+ (-0.14333511784309005) [Z4]
+ (0.0635910339418741) [Z3]
+ (0.06359103394187421) [Z2]
+ (0.12111503176024321) [Z0]
+ (0.12111503176024331) [Z1]
+ (-3.2142223089043954e-08) [Y1 Y3]
+ (-3.2142223089043954e-08) [X1 X3]
+ (2.8583487677736662e-08) [Y0 Y2]
+ (2.8583487677736662e-08) [X0 X2]
+ (3.058345114789404e-08) [Y5 Y7]
+ (3.058345114789404e-08) [X5 X7]
+ (5.4224294522282716e-08) [Y4 Y6]
+ (5.4224294522282716e-08) [X4 X6]
+ (0.00031259622325041354) [Y2 Y4]
+ (0.00031259622325041354) [X2 X4]
+ (0.0038017750347166136) [Y3 Y5]
+ (0.0038017750347166136) [X3 X5]
+ (0.08346186127902747) [Z0 Z2]
+ (0.08346186127902747) [Z1 Z3]
+ (0.09129117225478453) [Z2 Z6]
+ (0.09129117225478453) [Z3 Z7]
+ (0.09299736444454987) [Z0 Z6]
+ (0.09299736444454987) [Z1 Z7]
+ (0.09617293025640639) [Z2 Z7]
+ (0.09617293025640639) [Z3 Z6]
+ (0.098188560548433) [Z0 Z7]
+ (0.098188

In [8]:
ncore_orbs = 3
core = list(range(ncore_orbs))
core

[0, 1, 2]

In [12]:
orbitals = 5
active = list(range(ncore_orbs, orbitals))
active

[3, 4]